## Homework 10

In [1]:
# useful imports 
import tensorflow as tf
from tensorflow_text import RegexSplitter
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, AveragePooling2D, TimeDistributed, LSTM, GlobalAvgPool2D, AbstractRNNCell, MaxPooling2D, RNN
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict
import datetime
import tqdm

2023-02-09 00:05:24.990041: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 00:05:25.107774: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-09 00:05:25.110602: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-09 00:05:25.110612: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

### 2.2 Word Embeddings

In [2]:
# open the txt file
bible = open("bible.txt", "r")  
# read file
data = bible.read()  

In [3]:
# global hyperparameters
VOCAB_SIZE = 10000
WINDOW_SIZE = 4
BATCH_SIZE = 32

In [4]:
# convert to lower case and replace new-line characters with <space>
data = data.lower().replace("\n", " ")
# delete special characters, only alphanumeric values remain
# do we care for numbers or should we also delete them?
data = re.sub('\W+',' ', data)
# split on space
tokenized_data = data.split(' ')

In [5]:
def wordcount(data):
    uniqe_words = set(data)
    occurrences = defaultdict(lambda: 0)
    for item in tokenized_data:
        occurrences[item] +=1
    # sort occurences such that we can get the words with the highest counts
    # sorted from small to high counts
    sorted_occurrences = {k: v for k, v in sorted(occurrences.items(), key=lambda item: item[1])}
    most_common_words = list(sorted_occurrences.keys())[(12744-VOCAB_SIZE):]
    least_common_words = list(sorted_occurrences.keys())[:(12744-VOCAB_SIZE)+1]
    most_common_words = most_common_words[::-1]
    return most_common_words, least_common_words

In [ ]:
m_c_w, l_c_w = wordcount(tokenized_data)

# remove instances of least commen words from data (they will not be included??)
for word in l_c_w:
    data = re.sub(f' {word} ',' ', data)
    
# assign new tokenized data
tokenized_data = data.split(' ')

In [ ]:
# create a vocabulary for later purposes
vocab, index = {}, 1  # start indexing from 1

for word in m_c_w:
    vocab[word] = index
    index += 1
    
example_sequence = [vocab[word] for word in tokenized_data[:10]]
print(example_sequence)
print(tokenized_data[20:30])

[1, 253, 449, 3, 161, 193, 43, 43, 6, 1]
['and', 'the', 'earth', 'was', 'without', 'form', 'and', 'void', 'and', 'darkness']


In [ ]:
# inverse vocabulary
inverse_vocab = {index: token for token, index in vocab.items()}

In [ ]:
# to create input-target pairs (= word-contexts pairs) we use 
# tf's continuous skipgram model 
# https://www.tensorflow.org/tutorials/text/word2vec

bible_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      tokenized_data,
      vocabulary_size=VOCAB_SIZE,
      window_size=WINDOW_SIZE,
      negative_samples=0)

bible_skip_grams = [[vocab[x[0]], vocab[x[1]]] for x in bible_skip_grams]
inputs, targets = map(list, zip(*bible_skip_grams))
print(targets[:100])


[2, 2, 41, 1, 17, 46, 28, 126, 2, 5, 7, 3, 1199, 328, 292, 5, 705, 509, 168, 1, 13, 4477, 2, 203, 213, 12, 708, 11, 1, 109, 2, 53, 18, 38, 128, 1, 148, 34, 53, 1220, 1109, 53, 529, 18, 68, 28, 2, 1, 103, 4, 2, 1761, 24, 392, 9, 31, 20, 193, 31, 3, 10, 159, 139, 1, 1, 1217, 736, 1986, 19, 14, 58, 6, 2180, 72, 1436, 9, 216, 103, 100, 1, 3016, 71, 1970, 1, 280, 3883, 134, 26, 320, 9, 505, 61, 169, 31, 3, 5, 2, 102, 3, 2]


In [ ]:
# the usual cache, shuffle, batch, prefetch
bible_dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(1000).batch(32)


2023-02-09 00:06:24.642583: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-09 00:06:24.642605: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-09 00:06:24.642627: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (maries-debian): /proc/driver/nvidia/version does not exist
2023-02-09 00:06:24.642916: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2.3 The Model

In [15]:
class SkipGramModel(tf.keras.layers.Layer):
    def __init__(self, optimizer, embedding_size):
        super(SkipGramModel,self).__init__()
        
        self.vocabulary_size = VOCAB_SIZE
        self.embedding_size = embedding_size
        self.loss_metric = tf.keras.metrics.Mean(name = "loss")
        self.optimizer = optimizer
        
    def build(self,string):
        self.embedding_mat = self.add_weight(shape=(self.vocabulary_size, self.embedding_size),
                                             initializer="uniform",
                                             trainable=True) 
        self.output_mat = self.add_weight(shape=(self.vocabulary_size, self.embedding_size),
                                          initializer="random_normal",
                                          trainable=True) 
        self.output_bias = self.add_weight(shape=(self.vocabulary_size,),
                                           initializer="zero",
                                           trainable=True)
    def call(self, inputs):
        target_predicted = tf.nn.embedding_lookup(params=self.embedding_mat, ids=inputs)
        return target_predicted
    
    def train(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs)
            loss = tf.reduce_mean(
                tf.nn.nce_loss(weights=self.output_mat,
                               biases=self.output_bias, 
                               labels=tf.expand_dims(targets,axis=1), 
                               inputs=predictions,
                               num_sampled=1,
                               num_classes=self.vocabulary_size))
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # update loss metric
        self.metrics[0].update_state(loss)
        
        # update accuracy
        for metric in self.metrics[1:]:
            metric.update_state(targets,predictions)
        

In [16]:
def training_loop(model, train_ds, epochs, summary_writer):

    # iterate over epochs
    for epoch in range(epochs):
        print('Epoch: ', epoch)

        for data in train_ds:
            metrics = model.train(data)

        # log train loss
        with summary_writer.as_default():  
            # for scalar metrics:
            for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=e)


       
        model.reset_metrics()

In [17]:
def create_summary_writers(config_name):
    
    # Define where to save the logs
    # along with this, you may want to save a config file with the same name so you know what the hyperparameters were used
    # alternatively make a copy of the code that is used for later reference
    
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    train_log_path = f"logs/{config_name}/{current_time}/train"
    test_log_path = f"logs/{config_name}/{current_time}/test"

    # log writer for training metrics
    train_summary_writer = tf.summary.create_file_writer(train_log_path)

    # log writer for test metrics
    test_summary_writer = tf.summary.create_file_writer(test_log_path)
    
    return train_summary_writer, test_summary_writer

In [ ]:
# define model
EPOCHS = 2
optimizer = tf.keras.optimizers.Adam()
model = SkipGramModel(optimizer, 64)
summary_writer = create_summary_writers("model1")
training_loop(model, train_ds=bible_dataset, epochs= EPOCHS, summary_writer=summary_writer)

Epoch:  0
